# Dataset Exploration

In [1]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
print(f'Dataset: {dataset}')
print(f'===================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Number of classes: {dataset[0]}')

Dataset: KarateClub()
Number of graphs: 1
Number of features: 34
Number of classes: 4
Number of classes: Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])


In [2]:
from torch_sparse import SparseTensor
row, col = dataset[0].edge_index
N = dataset[0].num_nodes

adj = SparseTensor(row=row, col=col, sparse_sizes=(N, N))
print(row.shape, col.shape)

# dataset[0].edge_index
# adj
# set_diag(adj)

torch.Size([156]) torch.Size([156])


In [3]:
type(dataset[0].edge_index)

torch.Tensor

In [4]:
data = dataset[0]
data

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

# GAT Model Creation

In [5]:
from Classes.GAT import GATConv
import os
path = os.getcwd()
print(path)

/home/vimukthi/MyProjects/Machine-Learning/Graph_ML
/home/vimukthi/MyProjects/Machine-Learning/Graph_ML


In [17]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class GAT(nn.Module):
    def __init__(self, in_features, out_features, heads=1):
        super().__init__()
        self.conv1 = GATConv(in_features, out_features, heads)
        self.conv2 = GATConv(heads * out_features, dataset.num_classes, heads)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [18]:
model = GAT(in_features=34, out_features=16, heads=8)
print(model)

GAT(
  (conv1): GATConv(34, 16, heads=8)
  (conv2): GATConv(128, 4, heads=8)
)


In [8]:
data

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
lossArr = []
num_epochs = 1000

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(num_epochs):
    loss = train()
    lossArr.append(loss.item())
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 3.4433
Epoch: 001, Loss: 3.3605
Epoch: 002, Loss: 3.1892
Epoch: 003, Loss: 3.1411
Epoch: 004, Loss: 3.0091
Epoch: 005, Loss: 2.9612
Epoch: 006, Loss: 2.5060
Epoch: 007, Loss: 2.3481
Epoch: 008, Loss: 2.4253
Epoch: 009, Loss: 2.0986
Epoch: 010, Loss: 1.7293
Epoch: 011, Loss: 1.4397
Epoch: 012, Loss: 1.4019
Epoch: 013, Loss: 1.1393
Epoch: 014, Loss: 1.4261
Epoch: 015, Loss: 1.2740
Epoch: 016, Loss: 1.2167
Epoch: 017, Loss: 0.9581
Epoch: 018, Loss: 1.1937
Epoch: 019, Loss: 1.0473
Epoch: 020, Loss: 0.7270
Epoch: 021, Loss: 1.1569
Epoch: 022, Loss: 1.0017
Epoch: 023, Loss: 0.7754
Epoch: 024, Loss: 0.5664
Epoch: 025, Loss: 0.6042
Epoch: 026, Loss: 0.9117
Epoch: 027, Loss: 0.7419
Epoch: 028, Loss: 0.5654
Epoch: 029, Loss: 0.4837
Epoch: 030, Loss: 0.6156
Epoch: 031, Loss: 0.3224
Epoch: 032, Loss: 0.5012
Epoch: 033, Loss: 0.3889
Epoch: 034, Loss: 0.7309
Epoch: 035, Loss: 0.6402
Epoch: 036, Loss: 0.6670
Epoch: 037, Loss: 0.4127
Epoch: 038, Loss: 0.3118
Epoch: 039, Loss: 0.6882


## Original GAT

In [10]:
from torch_geometric.nn import GATConv

class GAT(nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GATConv(data.num_features, hidden_channels, heads)
        self.conv2 = GATConv(heads * hidden_channels, dataset.num_classes, heads)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x
    
model = GAT(hidden_channels=16, heads=8)    
print(model)

GAT(
  (conv1): GATConv(34, 16, heads=8)
  (conv2): GATConv(128, 4, heads=8)
)


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
lossArr = []
num_epochs = 10

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(num_epochs):
    loss = train()
    lossArr.append(loss.item())
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 3.4951
Epoch: 001, Loss: 3.3172
Epoch: 002, Loss: 3.2252
Epoch: 003, Loss: 3.0916
Epoch: 004, Loss: 3.0681
Epoch: 005, Loss: 2.8321
Epoch: 006, Loss: 2.6403
Epoch: 007, Loss: 2.6831
Epoch: 008, Loss: 2.2614
Epoch: 009, Loss: 2.2220
